# Assignment 5

In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import math
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

## Data pre-processing steps

In [24]:
df = pd.read_csv('student-por.csv')
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [25]:
df['romantic'].value_counts() #Let's have a look at the 'romantic' variable

no     410
yes    239
Name: romantic, dtype: int64

## Selecting variables
I selected these variables below for the variables and created a subset.

In [59]:
df_subset = df[['sex', 'age', 'freetime', 'absences', 'failures', 'goout', 'studytime', 'romantic']] #create subset with x variables to predict y.
df_subset.head() 

,sex,age,freetime,absences,failures,goout,studytime,romantic
0,F,18,3,4,0,4,2,no
1,F,17,3,2,0,3,2,no
2,F,15,3,6,0,2,2,no
3,F,15,2,0,0,2,3,yes
4,F,16,3,0,0,2,2,no


I created dummy variables for the variable sex.

In [60]:
#Creating dummy variables of qualitative variables
dummies = pd.get_dummies(df_subset['sex'])
df_subset = pd.concat([df_subset, dummies], axis=1) #the axis=1 means: add it to the columns (axis=0 is rows)
df_subset.head()

,sex,age,freetime,absences,failures,goout,studytime,romantic,F,M
0,F,18,3,4,0,4,2,no,1,0
1,F,17,3,2,0,3,2,no,1,0
2,F,15,3,6,0,2,2,no,1,0
3,F,15,2,0,0,2,3,yes,1,0
4,F,16,3,0,0,2,2,no,1,0


In [61]:
df_subset = df_subset[['F', 'age', 'freetime', 'absences', 'failures', 'goout', 'studytime','romantic']] #create subset with x variables to predict y.
df_subset.head()

,F,age,freetime,absences,failures,goout,studytime,romantic
0,1,18,3,4,0,4,2,no
1,1,17,3,2,0,3,2,no
2,1,15,3,6,0,2,2,no
3,1,15,2,0,0,2,3,yes
4,1,16,3,0,0,2,2,no


I split the data into a training and test set.

In [62]:
y = df_subset['romantic'] #We need to take out the romantic as our Y-variable
X =  df_subset[['F', 'age', 'freetime', 'absences', 'failures', 'goout', 'studytime']]#and the subset variables as our x variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) #split the data into training and test set, store it into different variables

## k-NN algorithm
The algorithm is a classification by finding the most similar data points in the training data, and making a guess based on their classifications. The algorithm picks the k nearest neighbor of the individual I want to predict for (so for this it is the variable romantic). So, for k I use 5. This means that it will look at the 5 closest neighbors of Y.

In [63]:
from sklearn.neighbors import KNeighborsClassifier

In [64]:
knn = KNeighborsClassifier(n_neighbors=5) # use k-nearest neighbor
knn.fit(X_train, y_train)
knn.score(X_test, y_test) #calculate the fit on the test data

0.6153846153846154

62% is predicted accurately if someone has a romantic relationship or not.

## Confusion matrix
I created the confusion matrix to see how well the model tells apart if someone has a romantic relationship or not.

In [65]:
from sklearn.metrics import confusion_matrix
y_test_pred = knn.predict(X_test) #the predicted values
cm = confusion_matrix(y_test, y_test_pred) #creates a "confusion matrix"
cm

array([[102,  26],
       [ 49,  18]], dtype=int64)

In [68]:
#In order to read it easily , let's make a dataframe out of it, and add labels to it.
conf_matrix = pd.DataFrame(cm, index=['No romantic', 'Romantic'], columns = ['No romantic (predicted)', 'Romantic (predicted)']) 
conf_matrix

,No romantic (predicted),Romantic (predicted)
No romantic,102,26
Romantic,49,18


So, 102 who have no romantic relationship are correctly predicted. 26 are instead wrong predicted. 

18 who have a romantic relationship are correctly predicted. 49 are instead wrong predicted. So, compared to the actual the wrong predicted is quite high.

The recall and precision for people who doesn't have a romantic relationship is:

$recall = \frac{102}{102 + 26} = .79$

$precision = \frac{102}{102 + 49} = .68$

Precision is about how precise the algorithm is. Here it is 68%. This is alright but a little low. Recall is about how much I can recall. Here it is 79% so this is good.